In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectConfig
from image_helpers import ImageHelpers
from tqdm.auto import tqdm
from metric_funcs import *

metric_func = Gaussian(MSE(), sigma=1)

cam_config = CameraConfig(position=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/world_mug.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/world_mug_sim.xml", cam_config)

object_init_position = (0, 1.3, 0.3)

random_orientations = np.random.uniform(0, 2 * np.pi, size=(10, 3))
for orient in random_orientations:
    img_world, obj_config = world_viewer.get_view_cropped(ObjectConfig(orient, object_init_position))
    random_orientations2 = np.random.uniform(0, 2 * np.pi, size=(1000, 3))
    lowest_loss = np.inf
    best_orient = None
    best_image = None

    for test_orient in tqdm(random_orientations2, leave=False):
        img_sim, orient_sim = sim_viewer.get_view_cropped(ObjectConfig(test_orient, obj_config.position))

        pad_shape = np.maximum(img_world.shape, img_sim.shape)
        img_world = ImageHelpers.pad_to_shape(img_world, pad_shape)
        img_sim = ImageHelpers.pad_to_shape(img_sim, pad_shape)

        loss = metric_func(img_world, img_sim)

        if loss < lowest_loss:
            lowest_loss = loss
            best_orient = orient_sim
            best_image = img_sim

    cv.imshow("real_img", img_world)
    cv.imshow("sim_img", best_image)
    cv.waitKey(0)

In [ ]:
cv.destroyAllWindows()